# Banxico SIE indexes

@roman

28 June, 2024

In [54]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from INEGIpy import Indicadores
from sie_banxico import SIEBanxico

from datetime import datetime
from dotenv import load_dotenv


In [ ]:
# load environment variables
load_dotenv()

# inegi class
inegi_api = Indicadores(token=os.getenv('API_INEGI'))


---
# InegiPy Exploration

## Indicadores

In [ ]:
# see bise catalogue
indicadores = inegi_api.catalogo_indicadores('BISE')

In [ ]:
# see bise catalogue
indicadores = inegi_api.catalogo_indicadores('BISE')

display(indicadores.head())

In [ ]:
series_values_list = [
    '6204482218', '6204482221', '6204482223', '6204482231'
]
indicadores.query("value.isin(@series_values_list)")

## Get INPC

### trial 1: '583731'

In [ ]:
# get inpc
df_inpc, metadatos = inegi_api.obtener_df(
    indicadores='583731',
    metadatos=True,

)
df_inpc

In [ ]:
inegi_api.consulta_metadatos(metadatos)

### trial 2: 

In [ ]:
# get inpc
df_inpc, metadatos = inegi_api.obtener_df(
    indicadores='628194',
    metadatos=True,

)
df_inpc, inegi_api.consulta_metadatos(metadatos)

### trial 3: granularity

[Here](https://www.inegi.org.mx/app/tabulados/default.aspx?nc=ca61_2018&idrt=137&opc=t) is the link to the INPC data by state, however, data is not easy to download. There are 55 cities.

In [ ]:
# get inpc
df_inpc, metadatos = inegi_api.obtener_df(
    indicadores='590684',
    metadatos=True,
)
df_inpc, inegi_api.consulta_metadatos(metadatos)

---
# Banxico SIE

## Minimum Wage

In [ ]:
list(minimum_wage_data.values())

In [61]:
# get data from Banxico
minimum_wage_data = {
    'no_border_north': 'SL11296',
    'general': 'SL11297',

}
banxico_api = SIEBanxico(
    token=os.getenv('API_BANXICO'),
    id_series=list(minimum_wage_data.values()),
    language='en'
)

In [ ]:
# see metadata
banxico_api.get_metadata()

In [ ]:
# consult data
today = datetime.today().strftime('%Y-%m-%d')
series_data = banxico_api.get_timeseries_range(
    init_date='2019-01-01',
    end_date=today
)
series_data

In [ ]:
dict_df = {}
for serie in series_data['bmx']['series']:
    df = pd.DataFrame(serie['datos'])
    df['date'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y')
    df.set_index('date', inplace=True)
    df.drop(columns=['fecha'], inplace=True)
    df.index = df.index.strftime('%Y-%m-%d')
    df = df.rename(columns={'dato': serie['idSerie']})
    dict_df[serie['idSerie']] = df

# join dataframes by index
df = pd.concat(dict_df.values(), axis=1)

# index to DateTimeIndex
df.index = pd.to_datetime(df.index)

# see df
df

---
# Sandbox

In [ ]:
?inegi_api.obtener_df